In [1]:
! py -m pip list

Package                                  Version
---------------------------------------- -------------------
absl-py                                  1.0.0
aiohttp                                  3.9.5
aiosignal                                1.3.1
altair                                   5.3.0
annotated-types                          0.6.0
anyio                                    3.5.0
argon2-cffi                              21.3.0
argon2-cffi-bindings                     21.2.0
asgiref                                  3.8.1
asttokens                                2.0.5
astunparse                               1.6.3
async-timeout                            4.0.3
asyncio                                  3.4.3
attrs                                    21.4.0
Babel                                    2.9.1
backcall                                 0.2.0
backoff                                  2.2.1
bcrypt                                   4.1.3
beautifulsoup4                           

In [2]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import pandas as pd
import os
import shutil
import numpy as np
import re
import sqlite3

In [9]:
OPENAI_DEPLOYMENT_ENDPOINT = "https://az-openai-document-question-answer-service.openai.azure.com/" 
OPENAI_API_KEY = "5d24331966b648738e5003caad552df8" 
OPENAI_API_VERSION = "2023-05-15"

OPENAI_DEPLOYMENT_NAME = "az-gpt_35_model"
OPENAI_MODEL_NAME="gpt-3.5-turbo"

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = "az-embedding_model" 
OPENAI_ADA_EMBEDDING_MODEL_NAME = "text-embedding-ada-002"

encoding_name = "cl100k_base"
# OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# CHROMA_PATH = "c:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\SERFF\\vector_db\\neil_db"
# data_folder = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\SERFF\\new_files"
csv_file = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\Parker\\vm-GenAI_BI\\Text to SQL_V1\\Nurse_notes_parker.csv"
### Cohere API key 

In [10]:
llm = AzureChatOpenAI(
                        temperature=0.1,
                        deployment_name=OPENAI_DEPLOYMENT_NAME,
                        model_name=OPENAI_MODEL_NAME,
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY            
                    )

embedding_model = AzureOpenAIEmbeddings(openai_api_key=OPENAI_API_KEY,
                                            deployment=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
                                            model=OPENAI_ADA_EMBEDDING_MODEL_NAME,
                                            azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT)

def get_embedding():
    return embedding_model
# data = load_documents()

In [11]:
# Load CSV into SQLite
def load_csv_to_sqlite(csv_file, db_file, table_name):
    conn = sqlite3.connect(db_file)
    df = pd.read_csv(csv_file, encoding='latin-1')
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()
    print(f"Data from {csv_file} loaded into SQLite table {table_name}.")

In [12]:
db_file = "nurse_notes.db"
table_name = "nurse_notes"

In [13]:
load_csv_to_sqlite(csv_file, db_file, table_name)

Data from C:\Users\SujaySunilNagvekar\VM\GEN AI\Parker\vm-GenAI_BI\Text to SQL_V1\Nurse_notes_parker.csv loaded into SQLite table nurse_notes.


In [14]:
import sqlite3
import pandas as pd
from tabulate import tabulate

# Reconnect to the SQLite database
conn = sqlite3.connect("nurse_notes.db")

# Query to fetch the top 10 rows from the Claims table
query = "SELECT * FROM nurse_notes LIMIT 10"

# Use pandas to execute the query and load the results into a DataFrame
df = pd.read_sql_query(query, conn)

# Print the DataFrame as a formatted table
print(tabulate(df, headers='keys', tablefmt='grid'))

# Close the connection
conn.close()


+----+---------------+--------------+-------------+-------+----------+------------------+------------------+-----------+---------+----------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
# Define a custom schema
def get_custom_table_schema():
    schema = {
        "Primary_key": "INTEGER (Structured)",
        "First_Name": "TEXT (Structured)",
        "Last_Name": "TEXT (Structured)",
        "Age": "INTEGER (Structured)",
        "Gender": "TEXT (Structured)",
        "Marital_Status": "TEXT (Structured)",
        "Race": "TEXT (Structured)",
        "Day": "TEXT (Structured)",
        "Shift": "TEXT (Structured)",
        "Nurse_name": "TEXT (Structured)",
        "Clinical_Notes": "TEXT (Unstructured)",
        "Non_clinical_notes": "TEXT (Unstructured)"
    }
    return "\n".join([f"{col}: {desc}" for col, desc in schema.items()])

In [41]:
import pandas as pd

# Define metadata for the table
table_metadata = pd.DataFrame({
    "Column_Name": [
        "Primary_key", "First_Name", "Last_Name", "Age", "Gender",
        "Marital_Status", "Race", "Day", "Shift", "Nurse_name",
        "Clinical_Notes", "Non_clinical_notes"
    ],
    "Column_Type": [
        "INTEGER", "varchar", "varchar", "INTEGER", "varchar",
        "varchar", "varchar", "varchar", "varchar", "varchar",
        "varchar", "varchar"
    ],
    "Description": [
        "Unique identifier for each record",
        "First name of the resident",
        "Last name of the resident",
        "Age of the resident",
        "Gender of the resident (Male/Female)",
        "Marital status of the resident (Single/Married/Widowed/Divorced)",
        "Race of the resident",
        "Day of the week the note was recorded",
        "Shift during which the note was recorded (Morning/Night)",
        "Name of the nurse who recorded the note",
        "Detailed clinical notes about the resident's incident or medical event/ emergency",
        "Non-clinical notes such as personal preferences or complaints"
    ]
})

# Display metadata
table_metadata


,Column_Name,Column_Type,Description
0,Primary_key,INTEGER,Unique identifier for each record
1,First_Name,varchar,First name of the resident
2,Last_Name,varchar,Last name of the resident
3,Age,INTEGER,Age of the resident
4,Gender,varchar,Gender of the resident (Male/Female)
5,Marital_Status,varchar,Marital status of the resident (Single/Married...
6,Race,varchar,Race of the resident
7,Day,varchar,Day of the week the note was recorded
8,Shift,varchar,Shift during which the note was recorded (Morn...
9,Nurse_name,varchar,Name of the nurse who recorded the note


In [42]:
table_name = "nurse_notes"
table_schema = get_custom_table_schema()

In [43]:
class ConversationManager:
    def __init__(self, history_limit=5):
        """
        Initialize the conversation manager with a history limit.
        """
        self.history = []
        self.history_limit = history_limit

    def update_conversation_history(self, question, answer):
        """
        Store previous questions and answers, but limit the history size.
        """
        self.history.append(f"Question: {question}\nAnswer: {answer}")
        if len(self.history) > self.history_limit:
            self.history.pop(0)  # Remove the oldest entry to keep the history within the limit

    def get_conversation_context(self):
        """
        Get the full conversation context as a string.
        """
        return self.history
    
    def clear_history(self):
        """
        Clear the conversation history to erase context.
        """
        self.history = []

    def reformulate_question(self, query_text, llm):
        """
        Reformulate the follow-up question based on the conversation history.
        """
               # Use the reformulation prompt to generate a standalone question
        reformulation_prompt = """
        Given a chat history ( User Questions and LLM response ) and the latest user question which might reference context in the chat history, 
        formulate a standalone question which can be understood without the chat history. Do NOT answer the question, 
        just reformulate it if needed and otherwise return it as is.

        Chat History:
        {chat_history}

        Latest Question:
        {latest_question}
        """

        # Prepare the input for the LLM model
        formatted_prompt = reformulation_prompt.format(
            chat_history=self.get_conversation_context(),
            latest_question=query_text
        )

        # Get the reformulated question from LLM
        reformulated_response = llm.invoke(input=formatted_prompt)
        standalone_question = reformulated_response.content.strip()

        return standalone_question

In [63]:
# Generate SQL Query using AzureChatOpenAI LLM
def generate_sql_query(llm, natural_language_query, table_metadata, table_name, schema):
    query_prompt = f"""
You are a SQL expert. Use the table schema provided below to generate a SQL query. The data is regarding a senior care living community. 

<table info>

Table: {table_name}
Schema:
{schema}
Metadata: 
{table_metadata}

<table info/>

<Special Instructions>
- The SQL query must extract data semantically from text-based columns like 'Clinical_Notes' , 'Non_Clinical_Notes'
without relying on exact pattern matching (e.g., LIKE or CONTAINS).
- Use only First Name for Nurses
<Special Instructions/>

Convert the following natural language query into a SQL query:

{natural_language_query}

Only return the SQL query, no additional text.
"""
    response = llm.invoke(query_prompt)
    return response.content

# Execute the SQL query
def execute_sql_query(db_file, sql_query):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()
    try:
        cursor.execute(sql_query)
        result = cursor.fetchall()
        columns = [desc[0] for desc in cursor.description]
        conn.close()
        return pd.DataFrame(result, columns=columns)
    except sqlite3.Error as e:
        conn.close()
        return f"SQL error: {e}"
    
# Generate Natural Language Response using LLM
def generate_response(llm, dataframe, standalone_question, chat_history):
    """
    Generate a natural language response based on the standalone question,
    the SQL query result, and the chat history. If no data is found, generate
    a response indicating the absence of relevant information in a specific manner,
    avoiding technical terms like 'database', 'query', or 'SQL'.
    """
    if dataframe.empty:
        # Use the LLM to generate a specific response for no data
        no_data_prompt = f"""
The following request was made, but the system could not find any relevant information. 
Provide a clear response to the user, avoiding technical terms like 'database', 'query', or 'SQL'. 
Focus on delivering an explanation in plain language.

User Request: {standalone_question}

Chat History:
{chat_history}
"""
        response = llm.invoke(no_data_prompt)
        return response.content.strip()
    else:
        # Convert the entire query result into a comma-separated string
        result_list = dataframe.astype(str).apply(lambda row: ", ".join(row), axis=1).tolist()
        result_text = "; ".join(result_list)

        # Use the LLM to generate a response based on the query result and chat history
        response_prompt = f"""
Based on the information retrieved and the previous conversation context, provide a clear, concise explanation 
to address the user's request. Avoid using technical terms like 'database', 'query', or 'SQL'. 
Focus on explaining the information naturally and in plain language.

User Request: {standalone_question}

Chat History:
{chat_history}

Information Retrieved:
{result_text}

<Special Instructions>:
- Always mention Resident's names whenever possible. Combine First Name and Last Name for Resident names

<Special Instructions/>

"""
        response = llm.invoke(response_prompt)
        return response.content.strip()






In [64]:
# Initialize the ConversationManager with a history limit of 5
conv_manager = ConversationManager(history_limit=3)


In [65]:
# Step 3: Get user query and convert it
user_question = "What is the Age of Sarah Thompson"

# Reformulate the question (if needed)

standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Age FROM nurse_notes WHERE First_Name = 'Sarah' AND Last_Name = 'Thompson'
Query Result:
   Age
0   75
According to the information I found, Sarah Thompson is 75 years old.


In [66]:
# Step 3: Get user query and convert it
user_question = "Did any Residents have Bleeding incidents in the Clinical notes? If yes, name those Residents "
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT DISTINCT First_Name, Last_Name
FROM nurse_notes
WHERE Clinical_Notes LIKE '%bleeding%'
Query Result:
  First_Name Last_Name
0      James    Miller
Sure, I can help you with that. I found two Residents who had bleeding incidents in their Clinical notes. Their names are James Miller. Is there anything else I can assist you with?


In [67]:
conv_manager.history

['Question: What is the age of a person named Sarah Thompson?\nAnswer: According to the information I found, Sarah Thompson is 75 years old.',
 'Question: Can you provide a list of Residents who had bleeding incidents in the Clinical notes?\nAnswer: Sure, I can help you with that. I found two Residents who had bleeding incidents in their Clinical notes. Their names are James Miller. Is there anything else I can assist you with?']

In [68]:
# Step 3: Get user query and convert it
user_question = "Describe James' clinical notes in detail"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes FROM nurse_notes WHERE First_Name = 'James' AND Last_Name = 'Miller'
Query Result:
                                      Clinical_Notes
0  Nurse_Note: Approximately 7:30 pm Nurse respon...
1  Nurse_Note: Primary CNA was undressing residen...
Sure, I can help you with that. Based on the information I found, James Miller had two clinical notes. The first note was about a fall incident where he was found sitting on the floor with his foot extended in front of the nurse's station. He was alert but confused and was unable to express what happened. The nurse assisted him to transfer to a wheelchair and then to his bed. The second note was about a skin tear on his left lower leg caused by the hook of the urinary drainage bag during undressing. There was small bleeding noted, but he denied pain.


In [69]:
conv_manager.history

['Question: What is the age of a person named Sarah Thompson?\nAnswer: According to the information I found, Sarah Thompson is 75 years old.',
 'Question: Can you provide a list of Residents who had bleeding incidents in the Clinical notes?\nAnswer: Sure, I can help you with that. I found two Residents who had bleeding incidents in their Clinical notes. Their names are James Miller. Is there anything else I can assist you with?',
 "Question: What are the details of James Miller's clinical notes?\nAnswer: Sure, I can help you with that. Based on the information I found, James Miller had two clinical notes. The first note was about a fall incident where he was found sitting on the floor with his foot extended in front of the nurse's station. He was alert but confused and was unable to express what happened. The nurse assisted him to transfer to a wheelchair and then to his bed. The second note was about a skin tear on his left lower leg caused by the hook of the urinary drainage bag duri

In [70]:
conv_manager.clear_history()

In [71]:
# Step 3: Get user query and convert it
user_question = "How many Notes did Nurse Olivia take"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT COUNT(*) FROM nurse_notes WHERE Nurse_name LIKE 'Olivia%'
Query Result:
   COUNT(*)
0         3
Nurse Olivia has taken 3 Notes.


In [72]:
# Step 3: Get user query and convert it
user_question = "Summarize all 3 of her Clinical Notes"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes FROM nurse_notes WHERE Nurse_name LIKE 'Olivia%' LIMIT 3
Query Result:
                                      Clinical_Notes
0  Nurse_Note: Primary CNA was undressing residen...
1  Nurse_Note: Safely You called to apprise that ...
2  Nurse_Note: During care resident's aide observ...
Nurse Olivia has taken 3 clinical notes about Resident's care. The first note is about a skin tear on Resident's left lower leg caused by the urinary drainage bag getting caught while the CNA was undressing Resident. The second note is about Resident looking for her watch and moving towards the floor, but she did not fall. The third note is about dry skin on Resident's right leg with surrounding skin discoloration and dryness. The resident was unable to say how she got it due to her cognitive deficit.


In [73]:
# Step 3: Get user query and convert it
user_question = "Were there any injuries in any of her notes"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes FROM nurse_notes WHERE Nurse_name LIKE 'Olivia%' AND Clinical_Notes LIKE '%injury%'
Query Result:
Empty DataFrame
Columns: [Clinical_Notes]
Index: []
Yes, Nurse Olivia's clinical notes mention three injuries. The first injury is a skin tear on the Resident's left lower leg caused by the urinary drainage bag getting caught while the CNA was undressing Resident. The second note is about Resident looking for her watch and moving towards the floor, but she did not fall. The third note is about dry skin on Resident's right leg with surrounding skin discoloration and dryness. The resident was unable to say how she got it due to her cognitive deficit.


In [74]:
conv_manager.clear_history()

In [75]:
# Step 3: Get user query and convert it
user_question = "Summarize Notes for Robert Nash, only Clinical"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes FROM nurse_notes WHERE First_Name = 'Robert' AND Last_Name = 'Nash'
Query Result:
                                      Clinical_Notes
0  Nurse_Note: Nurse responded to a call light in...
Based on the information retrieved, it appears that Robert Nash fell while sitting on the floor next to his bed. He was last seen sitting in his recliner chair and denied being in pain. His vital signs were normal, and he did not appear to have any injuries. His wife was informed, and their son was called about the fall. Dr. Kobylarz was also made aware of the situation.


In [76]:
conv_manager.clear_history()

In [77]:
# Step 3: Get user query and convert it
user_question = "Summarize Notes where skin tear was observed"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes, Non_clinical_notes 
FROM nurse_notes 
WHERE Clinical_Notes LIKE '%skin tear%' OR Non_clinical_notes LIKE '%skin tear%';
Query Result:
                                      Clinical_Notes  \
0  Nurse_Note: Primary CNA was undressing residen...   
1  Nurse_Note: 5:50pm I was in room 35 assisting ...   

                                  Non_clinical_notes  
0  Nurse_Note: At 11:00 am on 09/20/2023 nurse wa...  
1  Nurse_Note: At 5:30 pm on 02/05/2024 nurse was...  
Based on the notes, there were three instances of observed skin tears. The first one was caused by the hook of the urinary drainage bag getting caught on the left lower leg front area while the resident was being undressed. The second one was observed on the left lateral aspect of the knee and left lateral elbow after the resident fell and was transferred to RWJUH. The third one was observed on an unspecified area. In all instances, the resident denied pain or discomfort.


In [ ]:
# Step 3: Get user query and convert it
user_question = "what were the names of the residents with skin tear"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT DISTINCT First_Name, Last_Name 
FROM nurse_notes 
WHERE Clinical_Notes LIKE '%skin tear%' OR Non_clinical_notes LIKE '%skin tear%'
Query Result:
  First_Name Last_Name
0      James    Miller
1     Olivia  Martinez
Based on the notes, two residents experienced skin tears. The first resident is James Miller and the second resident is Olivia Martinez.


In [201]:
conv_manager.clear_history()
conv_manager.history

[]

In [202]:
# Step 3: Get user query and convert it
user_question = "Summarize any incidents with fall or slip , Name the residents as well"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT First_Name, Last_Name
FROM nurse_notes
WHERE Clinical_Notes LIKE '%fall%' OR Clinical_Notes LIKE '%slip%'
Query Result:
  First_Name Last_Name
0      James    Miller
1    Michael   Johnson
2    Jessica     Brown
3     Robert      Nash
Sure, I can provide you with the names of the residents involved in any fall or slip incidents that have occurred. The residents are James Miller, Michael Johnson, Jessica Brown, and Robert Nash. Would you also like a summary of these incidents?


In [203]:
# Step 3: Get user query and convert it
user_question = "Yes , Explain all their notes in detail"
# Reformulate the question (if needed)
standalone_question = conv_manager.reformulate_question(user_question, llm)

sql_query = generate_sql_query(llm, table_metadata, standalone_question, table_name, table_schema)
print(f"Generated SQL Query: {sql_query}")

# Step 4: Execute the SQL query
query_result = execute_sql_query(db_file, sql_query)
if isinstance(query_result, str):  # Error occurred
        print(query_result)
else:
        print("Query Result:")
        print(query_result)

        # Step 5: Generate natural language response
        query_response = generate_response(llm, query_result,standalone_question, conv_manager.get_conversation_context())
        print(query_response)
conv_manager.update_conversation_history(standalone_question, query_response)

Generated SQL Query: SELECT Clinical_Notes, Non_clinical_notes 
FROM nurse_notes 
WHERE (Clinical_Notes LIKE '%fall%' OR Clinical_Notes LIKE '%slip%') 
OR (Non_clinical_notes LIKE '%fall%' OR Non_clinical_notes LIKE '%slip%')
Query Result:
                                      Clinical_Notes  \
0  Nurse_Note: Approximately 7:30 pm Nurse respon...   
1  Nurse_Note: Safely You called to apprise that ...   
2  Nurse_Note: It was reported to me by the Nurse...   
3  Nurse_Note: Nurse responded to a call light in...   

                                  Non_clinical_notes  
0  Nurse_Note: At 2:30 pm on 08/12/2023 nurse was...  
1  Nurse_Note: At 8:45 am on 11/23/2023 nurse was...  
2  Nurse_Note: At 10:30 am on 12/01/2023 nurse wa...  
3  Nurse_Note: At 3:00 pm on 12/15/2023 nurse was...  
Sure, I can provide you with the notes related to the residents involved in fall or slip incidents. The residents are James Miller, Michael Johnson, Jessica Brown, and Robert Nash. For James Miller, a nur